In [1]:
import cv2
import mmcv
import os
import numpy as np
from mmcv.transforms import Compose
from mmengine.utils import track_iter_progress
from mmdet.registry import VISUALIZERS
from mmdet.apis import init_detector, inference_detector
from tqdm import tqdm

/home/yaniv.sheinfeld/.pyenv/versions/3.9.4/envs/venv39/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def get_gt(filename):
    #if filename ends with ng then it is a positive sample
    #if filename ends with ok then it is a negative sample
    if filename.endswith('ng.jpg'):
        return 1
    elif filename.endswith('ok.jpg'):
        return 0

def get_tp_fp_tn_fn(pred, gt):
    if pred == 1 and gt == 1:
        return 1, 0, 0, 0
    elif pred == 1 and gt == 0:
        return 0, 1, 0, 0
    elif pred == 0 and gt == 0:
        return 0, 0, 1, 0
    elif pred == 0 and gt == 1:
        return 0, 0, 0, 1
    else:
        raise Exception('invalid input')

In [23]:
config_file = 'configs/je/mask-rcnn_r50-caffe_fpn_ms-poly-1x_coco_je.py'
#checkpoint_file = 'work_dirs/mask-rcnn_r50-caffe_fpn_ms-poly-1x_coco_je/epoch_1.pth'
test_path = '/home/yaniv.sheinfeld/data/TEST_SET_6.6.6/201/images'
#build the model from a config file and a checkpoint file
#model = init_detector(config_file, checkpoint_file, device='cuda:0')

imgs_list = []
im_paths = [os.path.join(test_path, im) for im in os.listdir(test_path)]    
thresholds = np.arange(0.1, 1.0, 0.1)
#for each threshold, calculate Detection Rate and False Positive Rate on all images in test_path
for epoch in range(1,31):
    checkpoint_file = f'work_dirs/mask-rcnn_r50-caffe_fpn_ms-poly-1x_coco_je/epoch_{epoch}.pth'
    model = init_detector(config_file, checkpoint_file, device='cuda:0')
    for threshold in thresholds:
        print(f'epoch: {epoch}, threshold: {threshold}')
        #init counters
        TP = 0
        FP = 0
        TN = 0
        FN = 0
        
        results = inference_detector(model, im_paths)
        pred = any(result.pred_instances.scores > threshold)
        #get the ground truth from the image path
        gt = get_gt(im_path)
        #calculate the number of true positives, false positives and false negatives
        tp, fp, tn, fn = get_tp_fp_tn_fn(pred, gt)
        TP += tp
        FP += fp
        TN += tn
        FN += fn
        #calculate Detection Rate and False Positive Rate
        DR = TP / (TP + FN)
        FPR = FP / (FP + TN)
        print('DR: ', DR)
        print('FPR: ', FPR)
        print('')
        if DR < 0.95:
            break

100%|██████████| 873/873 [00:09<00:00, 87.53it/s]


Loads checkpoint by local backend from path: work_dirs/mask-rcnn_r50-caffe_fpn_ms-poly-1x_coco_je/epoch_6.pth
epoch: 6, threshold: 0.1


AttributeError: 'list' object has no attribute 'pred_instances'

In [24]:
# Specify the path to model config and checkpoint file
config_file = 'configs/je/mask-rcnn_r50-caffe_fpn_ms-poly-1x_coco_je.py'
#checkpoint_file = 'work_dirs/mask-rcnn_r50-caffe_fpn_ms-poly-1x_coco_je/epoch_1.pth'
test_path = '/home/yaniv.sheinfeld/data/TEST_SET_6.6.6/201/images'
#build the model from a config file and a checkpoint file
#model = init_detector(config_file, checkpoint_file, device='cuda:0')

imgs_list = []
for im in os.listdir(test_path):
    #read image with cv2
    im_path = os.path.join(test_path, im)
    img = cv2.imread(im_path, cv2.IMREAD_COLOR)
    #append image to list
    imgs_list.append(img)

#init visualizer
#visualizer = VISUALIZERS.build(model.cfg.visualizer)

#the dataset_meta is loaded from the checkpoint file and then pass to the model in init_detector
#visualizer.dataset_meta = model.dataset_meta

#im_path = '/home/yaniv.sheinfeld/data/TEST_SET_6.6.6/201/images/20221124_082931_791875_201_ng.jpg'
#result = inference_detector(model, im_path)
thresholds = np.arange(0.1, 1.0, 0.1)
#for each threshold, calculate Detection Rate and False Positive Rate on all images in test_path
for epoch in range(6,31):
    checkpoint_file = f'work_dirs/mask-rcnn_r50-caffe_fpn_ms-poly-1x_coco_je/epoch_{epoch}.pth'
    model = init_detector(config_file, checkpoint_file, device='cuda:0')
    for threshold in thresholds:
        print(f'epoch: {epoch}, threshold: {threshold}')
        #init counters
        TP = 0
        FP = 0
        TN = 0
        FN = 0
        #for each image in test_path
        for im in tqdm(os.listdir(test_path), desc='test'):
            #inference
            im_path = os.path.join(test_path, im)
            results = inference_detector(model, im_path)
            pred = any(result.pred_instances.scores > threshold)
            #get the ground truth from the image path
            gt = get_gt(im_path)
            #calculate the number of true positives, false positives and false negatives
            tp, fp, tn, fn = get_tp_fp_tn_fn(pred, gt)
            TP += tp
            FP += fp
            TN += tn
            FN += fn
        #calculate Detection Rate and False Positive Rate
        DR = TP / (TP + FN)
        FPR = FP / (FP + TN)
        print('DR: ', DR)
        print('FPR: ', FPR)
        print('')
        if DR < 0.95:
            break


Loads checkpoint by local backend from path: work_dirs/mask-rcnn_r50-caffe_fpn_ms-poly-1x_coco_je/epoch_6.pth
epoch: 6, threshold: 0.1


test: 100%|██████████| 873/873 [00:58<00:00, 14.82it/s]


DR:  0.9939393939393939
FPR:  0.559322033898305

epoch: 6, threshold: 0.2


test: 100%|██████████| 873/873 [00:59<00:00, 14.75it/s]


DR:  0.9454545454545454
FPR:  0.3432203389830508

Loads checkpoint by local backend from path: work_dirs/mask-rcnn_r50-caffe_fpn_ms-poly-1x_coco_je/epoch_7.pth
epoch: 7, threshold: 0.1


...

DR:  0.9818181818181818
FPR:  0.5677966101694916

epoch: 30, threshold: 0.2


test: 100%|██████████| 873/873 [01:10<00:00, 12.43it/s]


DR:  0.9636363636363636
FPR:  0.423728813559322

epoch: 30, threshold: 0.30000000000000004


test: 100%|██████████| 873/873 [01:04<00:00, 13.48it/s]

DR:  0.9333333333333333
FPR:  0.3418079096045198

